Generate choicefile

In [1]:
trafficfilenamearr = ["unv1","prv1","cluster_a","cluster_b","cluster_c"]
numintervalarr = [4,4,24,24,24]
for itrafficfilename,trafficfilename in enumerate(trafficfilenamearr):
    numinterval = numintervalarr[itrafficfilename]
    choicefile = f"/home/annzhou/DRing/src/emp/datacentre/choicefiles/{trafficfilename}"
    with open(choicefile,'w') as f:
        for _ in range(numinterval):
            f.write("5\n")

Generate qivarfiles for meta
inspired by computerouting2/computerouting2.ipynb

In [4]:
def copy(copyfromfile, copyintofile):
    with open(copyfromfile,'r') as fr:
        lines = fr.readlines()
        with open(copyintofile,'w') as fw:
            for line in lines:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                pid = int(tokens[2])
                value = float(tokens[5])
                fw.write(f"{fromsw},{tosw},{pid},{value}\n")

In [5]:
clusterarr = ["a","b","c"]
intervalstart = 24
intervalend = 48
factor = 64
numsw = 80
racke = 0
step = 1800

for cluster in clusterarr:
    for interval in range(intervalstart, intervalend):
        copyfromivarfile = f"/home/annzhou/DRing/src/emp/datacentre/pathweightfiles/dring/racke{racke}/cluster{cluster}/pathweight_dring_racke{racke}_cluster{cluster}_lp1_barriernocrossover_{interval*step}_{(interval+1)*step}_{factor}.txt"
        copyintoqivarfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/qivar_dring_0_0_cluster_{cluster}_{interval}_racke{racke}_2_0"
        copy(copyfromivarfile, copyintoqivarfile)

Generate qivarfiles for wisc
- compute: inspired by computerouting2/testtemporalH_wisc.py
- quantize: inspired by makepathweightfiles/makepathweightfiles.ipynb

In [ ]:
import sys
import os
import gurobipy as gp
from gurobipy import GRB

trafficname = "prv1"
scaledown = 10000000
H = 1
homedir = "/home/annzhou"
routingname = "racke0"


def compute(interval,graphfile,numsw,netpathfile,traffic,method,crossover,qivarfile):
    # read netpathfile
    netpath = list()
    for i in range(numsw):
        netpath.append(list())
        for j in range(numsw):
            netpath[i].append(list())
    with open(netpathfile,'r') as f:
        lines = f.readlines()
        # produce
        fromsw = 0
        tosw = 0
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
            else:
                path = [fromsw]
                tokens = line.split()
                for token in tokens:
                    hops = token.split("->")
                    path.append(int(hops[1]))
                netpath[fromsw][tosw].append(path)

        # check
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                numpaths = int(tokens[2])
                if len(netpath[fromsw][tosw])!=numpaths:
                    print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

    # read graphfile
    link = list()
    for i in range(numsw):
        link.append(list())
        for j in range(numsw):
            link[i].append(0)
    with open(graphfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split("->")
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            link[fromsw][tosw] = 1
            link[tosw][fromsw] = 1

    # read linkfailurefile (if needed)
    # if numfaillink > 0:
    #     with open(linkfailurefile,'r') as f:
    #         lines = f.readlines()
    #         for line in lines:
    #             tokens = line.split()
    #             fromsw = int(tokens[0])
    #             tosw = int(tokens[1])
    #             if link[fromsw][tosw] != 1:
    #                 print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
    #             else:
    #                 link[fromsw][tosw] /= 2

    # read serverfile
    # serverdict = dict()
    # with open(serverfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(',')
    #         serverdict[int(tokens[0])] = int(tokens[1])

    # read flowfile
    # traffic = list()
    # for i in range(numsw):
    #     traffic.append(list())
    #     for j in range(numsw):
    #         traffic[i].append(0)
    # with open(flowfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(",")
    #         fromsvr = int(tokens[0])
    #         tosvr = int(tokens[1])
    #         if fromsvr>=numserver or tosvr>=numserver: continue
    #         fromsw = serverdict[fromsvr]
    #         tosw = serverdict[tosvr]
    #         if fromsw == tosw: continue
    #         flowbytes = int(tokens[2])
    #         traffic[fromsw][tosw] += flowbytes


    # precompute
    flowsvialink = list()
    for i in range(numsw):
        flowsvialink.append(list())
        for j in range(numsw):
            flowsvialink[i].append(list())
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                for pid,path in enumerate(netpath[fromsw][tosw]):
                    fidpidstr = f"{fromsw},{tosw},{pid}"
                    prevhop = fromsw
                    for hop in path[1:]:
                        flowsvialink[prevhop][hop].append(fidpidstr)
                        prevhop = hop

    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         traffic[interval][fromsw][tosw] /= 100000


    # Create a new model
    model = gp.Model("mcf")

    # Add variables
    maxpid = 0
    for fromsw in range(numsw):
        for tosw in range(numsw):
            maxpid = max(maxpid,len(netpath[fromsw][tosw]))
    vararr = list()
    for fromsw in range(numsw):
        vararr.append(list())
        for tosw in range(numsw):
            vararr[fromsw].append(list())
            for pid in range(maxpid):
                vararr[fromsw][tosw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                    vararr[fromsw][tosw][pid] = var
    k = model.addVar(name="k")

    # Set objective function
    model.setObjective(k, GRB.MAXIMIZE)

    # Add constraints
    # Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                varlist = list()
                for pid in range(len(netpath[fromsw][tosw])):
                    varlist.append(vararr[fromsw][tosw][pid])
                model.addConstr(sum(varlist)>=k*traffic[interval][fromsw][tosw],f"c0_{fromsw}_{tosw}")

    # Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
    for linkfrom in range(numsw):
        for linkto in range(numsw):
            flowstrlist = flowsvialink[linkfrom][linkto]
            if len(flowstrlist) > 0:
                varlist = list()
                for flowstr in flowstrlist:
                    tokens = flowstr.split(',')
                    flowfrom = int(tokens[0])
                    flowto = int(tokens[1])
                    pid = int(tokens[2])
                    varlist.append(vararr[flowfrom][flowto][pid])
                model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

    # Constraint 2: for each fid: for each pid: pid = prev_pid
    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         if traffic[interval][fromsw][tosw] > 0:
    #             for pid in range(1,len(netpath[fromsw][tosw])):
    #                 model.addConstr(vararr[fromsw][tosw][pid]-vararr[fromsw][tosw][pid-1]==0,f"c2_{fromsw}_{tosw}_{pid}")

    # Optimize model
    model.setParam('Method',method)
    model.setParam('Crossover',crossover)
    model.optimize()
    # model.write(modelfile)

    # Print results
    if model.status == GRB.OPTIMAL:
        print(f"Optimal objective value: {model.objVal}")
        with open(qivarfile,'a') as f:
            f.write(f"{routingname},{interval},{k.x}\n")
            for fromsw in range(numsw):
                for tosw in range(numsw):
                    if traffic[interval][fromsw][tosw] > 0:
                        for pid in range(len(netpath[fromsw][tosw])):
                            var = vararr[fromsw][tosw][pid]
                            f.write(f"{var.varName},{var.x}\n")
    else:
        print("No optimal solution found")
    return


def solve(computeinterval,computeqivarfile,solveinterval,graphfile,numsw,netpathfile,traffic,cachefile,method,crossover):
    if os.path.exists(cachefile):
        with open(cachefile,'r') as f:
            lines = f.readlines()
            for line in lines:
                tokens = line.split(',')
                mycomputeinterval = int(tokens[0])
                mysolveinterval = int(tokens[1])
                mythroughput = float(tokens[2])
                if computeinterval==mycomputeinterval and solveinterval==mysolveinterval:
                    return mythroughput

    if not os.path.exists(computeqivarfile):
        compute(computeinterval,graphfile,numsw,netpathfile,traffic,method,crossover,computeqivarfile)


    # read netpathfile
    netpath = list()
    for i in range(numsw):
        netpath.append(list())
        for j in range(numsw):
            netpath[i].append(list())
    with open(netpathfile,'r') as f:
        lines = f.readlines()
        # produce
        fromsw = 0
        tosw = 0
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
            else:
                path = [fromsw]
                tokens = line.split()
                for token in tokens:
                    hops = token.split("->")
                    path.append(int(hops[1]))
                netpath[fromsw][tosw].append(path)

        # check
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                numpaths = int(tokens[2])
                if len(netpath[fromsw][tosw])!=numpaths:
                    print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

    # read graphfile
    link = list()
    for i in range(numsw):
        link.append(list())
        for j in range(numsw):
            link[i].append(0)
    with open(graphfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split("->")
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            link[fromsw][tosw] = 1
            link[tosw][fromsw] = 1

    # read linkfailurefile (if needed)
    # if numfaillink > 0:
    #     with open(linkfailurefile,'r') as f:
    #         lines = f.readlines()
    #         for line in lines:
    #             tokens = line.split()
    #             fromsw = int(tokens[0])
    #             tosw = int(tokens[1])
    #             if link[fromsw][tosw] != 1:
    #                 print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
    #             else:
    #                 link[fromsw][tosw] /= 2

    # read serverfile
    # serverdict = dict()
    # with open(serverfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(',')
    #         serverdict[int(tokens[0])] = int(tokens[1])

    # read flowfile
    # traffic = list()
    # for i in range(numsw):
    #     traffic.append(list())
    #     for j in range(numsw):
    #         traffic[i].append(0)
    # with open(flowfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(",")
    #         fromsvr = int(tokens[0])
    #         tosvr = int(tokens[1])
    #         if fromsvr>=numserver or tosvr>=numserver: continue
    #         fromsw = serverdict[fromsvr]
    #         tosw = serverdict[tosvr]
    #         if fromsw == tosw: continue
    #         flowbytes = int(tokens[2])
    #         traffic[fromsw][tosw] += flowbytes

    # read computeqivarfile
    pathweight = list()
    for i in range(numsw):
        pathweight.append(list())
        for j in range(numsw):
            pathweight[i].append(dict())
    with open(computeqivarfile,'r') as f:
        lines = f.readlines()[1:]
        for line in lines:
            tokens = line.split(",")
            string = tokens[0]
            stringtokens = string.split("_")
            fromsw = int(stringtokens[0])
            tosw = int(stringtokens[1])
            pid = int(stringtokens[2])
            weight = float(tokens[1])
            pathweight[fromsw][tosw][pid] = weight
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw]>0 and traffic[computeinterval][fromsw][tosw]==0:
                numnetpath = len(netpath[fromsw][tosw])
                for i in range(numnetpath):
                    pathweight[fromsw][tosw][i] = 1.0/numnetpath


    # precompute
    flowsvialink = list()
    for i in range(numsw):
        flowsvialink.append(list())
        for j in range(numsw):
            flowsvialink[i].append(list())
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid,path in enumerate(netpath[fromsw][tosw]):
                    fidpidstr = f"{fromsw},{tosw},{pid}"
                    prevhop = fromsw
                    for hop in path[1:]:
                        flowsvialink[prevhop][hop].append(fidpidstr)
                        prevhop = hop

    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         traffic[interval][fromsw][tosw] /= 100000


    # Create a new model
    model = gp.Model("mcf")

    # Add variables
    maxpid = 0
    for fromsw in range(numsw):
        for tosw in range(numsw):
            maxpid = max(maxpid,len(netpath[fromsw][tosw]))
    vararr = list()
    for fromsw in range(numsw):
        vararr.append(list())
        for tosw in range(numsw):
            vararr[fromsw].append(list())
            for pid in range(maxpid):
                vararr[fromsw][tosw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                    vararr[fromsw][tosw][pid] = var
    k = model.addVar(name="k")
    avararr = list()
    for fromsw in range(numsw):
        avararr.append(list())
        for tosw in range(numsw):
            avararr[fromsw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                avar = model.addVar(name=f"a_{fromsw}_{tosw}")
                avararr[fromsw][tosw] = avar

    # Set objective function
    model.setObjective(k, GRB.MAXIMIZE)

    # Add constraints
    # Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                varlist = list()
                for pid in range(len(netpath[fromsw][tosw])):
                    varlist.append(vararr[fromsw][tosw][pid])
                model.addConstr(sum(varlist)>=k*traffic[solveinterval][fromsw][tosw],f"c0_{fromsw}_{tosw}")

    # Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
    for linkfrom in range(numsw):
        for linkto in range(numsw):
            flowstrlist = flowsvialink[linkfrom][linkto]
            if len(flowstrlist) > 0:
                varlist = list()
                for flowstr in flowstrlist:
                    tokens = flowstr.split(',')
                    flowfrom = int(tokens[0])
                    flowto = int(tokens[1])
                    pid = int(tokens[2])
                    varlist.append(vararr[flowfrom][flowto][pid])
                model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

    # Constraint 2: for each fid: for each pid: pid = pathweight[fid][pid] * a[fid]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    model.addConstr(vararr[fromsw][tosw][pid]-pathweight[fromsw][tosw][pid]*avararr[fromsw][tosw]==0,f"c2_{fromsw}_{tosw}_{pid}")

    # Optimize model
    model.setParam('Method',-1)
    model.setParam('Crossover',0)
    model.optimize()
    # model.write(modelfile)

    # Print results
    if model.status == GRB.OPTIMAL:
        print(f"Optimal objective value: {model.objVal}")
        with open(cachefile,'a') as f:
            f.write(f"{computeinterval},{solveinterval},{k.x}\n")
            # for fromsw in range(numsw):
            #     for tosw in range(numsw):
            #         if traffic[fromsw][tosw] > 0:
            #             for pid in range(len(netpath[fromsw][tosw])):
            #                 var = vararr[fromsw][tosw][pid]
            #                 f.write(f"{var.varName},{var.x}\n")
    else:
        print("No optimal solution found")
    return k.x


# set up parameters
# if trafficname == "cluster_b":
#     trafficfilesuffixlist = ["0_500","500_1000","1000_1500","1500_2000","2000_2500","2500_2900"]
# else:
#     trafficfilesuffixlist = ["0_273"]
# step_s = 1800
numserver = 2988
numsw = 80
numport = 64
# numinterval = 86400//step_s
numinterval = 8
graphname = "dring"
numfaillink = 0
numlink = 2132
fseed = 0
method = 2
crossover = 0
factor = 64

if graphname=="dring":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/double_ring/instance1_{numsw}_{numport}.edgelist"
elif graphname=="rrg":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/rrg/instance1_{numsw}_{numport}.edgelist"
else:
    print("ERROR: leafspine does not need to compute path weight")


# read in traffic
serverfile = f"{homedir}/DRing/src/emp/datacentre/serverfiles/dring_{numserver}_{numsw}_{numport}"
serverdict = dict()
with open(serverfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        serverdict[int(tokens[0])] = int(tokens[1])

traffic = list()
for k in range(numinterval):
    traffic.append(list())
    for i in range(numsw):
        traffic[k].append(list())
        for j in range(numsw):
            traffic[k][i].append(0)
# for suffix in trafficfilesuffixlist:
#     trafficfile = f"{homedir}/DRing/src/emp/datacentre/trafficfiles/{trafficname}/traffic_64racks_{suffix}"
#     with open(trafficfile,'r') as f:
#         lines = f.readlines()
#         for line in lines:
#             tokens = line.split()
#             time_s = int(tokens[0])
#             interval = time_s//step_s
#             fromsvr = int(tokens[1])
#             tosvr = int(tokens[2])
#             if fromsvr>=numserver or tosvr>=numserver: continue
#             fromsw = serverdict[fromsvr]
#             tosw = serverdict[tosvr]
#             if fromsw == tosw: continue
#             flowbytes = int(tokens[3])
#             traffic[interval][fromsw][tosw] += flowbytes
trafficfile = f"{homedir}/DRing/src/emp/datacentre/trafficfiles/{trafficname}"
with open(trafficfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        interval = int(tokens[0])
        fromsvr = int(tokens[1])
        tosvr = int(tokens[2])
        if fromsvr>=numserver or tosvr>=numserver: continue
        fromsw = serverdict[fromsvr]
        tosw = serverdict[tosvr]
        if fromsw == tosw: continue
        flowbytes = int(tokens[3])
        traffic[interval][fromsw][tosw] += flowbytes
for k in range(numinterval):
    for i in range(numsw):
        for j in range(numsw):
            traffic[k][i][j] /= scaledown

totaltraffic_per_interval = list()
for k in range(numinterval):
    totaltraffic = 0
    for i in range(numsw):
        for j in range(numsw):
            totaltraffic += traffic[k][i][j]
    totaltraffic_per_interval.append(totaltraffic)


# read in throughput with equal path weights
# equalthroughput = dict()
# for routingname in routingnamearr:
#     equalthroughput[routingname] = dict()
# with open(spatialfile,'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         tokens = line.split(',')
#         routingname = tokens[0]
#         interval = int(tokens[1])
#         throughput = float(tokens[2])
#         equalthroughput[routingname][interval] = throughput


# compute & solve
netpathfile = f"{homedir}/DRing/src/emp/datacentre/netpathfiles/netpath_{routingname}_{graphname}.txt"
for interval in range(3,7):
    computeqivarfile = f"{homedir}/DRing/src/emp/datacentre/qivarfiles/ivar_compute_{graphname}_{numfaillink}_{fseed}_{trafficname}_{interval}_{routingname}_{method}_{crossover}"
    compute(interval,graphfile,numsw,netpathfile,traffic,method,crossover,computeqivarfile)

In [17]:
# quantize
# copy from makepathweightfiles/makepathweightfiles.ipynb
import random
random.seed(0)

def quantize_to_multiple_of_1_64(x,factor):
    return round(x * factor) / factor

def quantize_and_adjust_with_indices(indexed_values,factor,isprint=False):
    if isprint: print(indexed_values)
    # Quantize each value
    quantized_values = [(i, quantize_to_multiple_of_1_64(v,factor)) for i, v in indexed_values]
    if isprint: print(quantized_values)
    
    # Calculate the current sum
    current_sum = sum(v for _, v in quantized_values)
    
    # Calculate the difference between the sum and 1
    difference = current_sum - 1
    if isprint: print(f"currsum={current_sum}, difference={difference}")

    random_values = [(v, random.random()) for v in quantized_values]
    if isprint: print(random_values)
    
    if difference != 0:
        if difference > 0:
            # Sort indices by quantized values to adjust the largest values first
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]), reverse=True)
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            if isprint: print(sorted_random)
            if isprint: print(sorted_quantized)
            
            while difference !=0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(difference, value % (1 / factor))
                    # adjustment = difference
                    new_value = value - 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference -= 1.0/factor
                
        else:
            # If the difference is negative (i.e., sum is less than 1), we need to increase values
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1])
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]))
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            
            while difference != 0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(-difference, (1 / factor) - (value % (1 / factor)))
                    # adjustment = -difference
                    new_value = value + 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference += 1.0/factor
    
    # Return the list sorted by original indices
    # return [v for i, v in sorted(quantized_values)]
    if isprint: print(quantized_values)
    return quantized_values

def check(qvarfile, numsw):
    # check qvarfile
    checkweightarr = list()
    for i in range(numsw):
        checkweightarr.append(list())
        for j in range(numsw):
            checkweightarr[i].append(0)
    with open(qvarfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split(',')
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            pid = int(tokens[2])
            weight = float(tokens[3])
            checkweightarr[fromsw][tosw] += weight

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if checkweightarr[fromsw][tosw]!=0 and checkweightarr[fromsw][tosw]!=1:
                print(f"from {fromsw} to {tosw}: weight={checkweightarr[fromsw][tosw]}")

def quantize(varfile, qvarfile, factor, numsw):
    with open(varfile,'r') as f:
        lines = f.readlines()[1:]

    valuearr = list()
    weightarr = list()
    qweightarr = list()
    for i in range(numsw):
        valuearr.append(list())
        weightarr.append(list())
        qweightarr.append(list())
        for j in range(numsw):
            valuearr[i].append(list())
            weightarr[i].append(list())
            qweightarr[i].append(list()) 

    for line in lines:
        tokens = line.split(',')
        varname = tokens[0]
        vartokens = varname.split('_')
        fromsw = int(vartokens[0])
        tosw = int(vartokens[1])
        pid = int(vartokens[2])
        value = float(tokens[1])
        valuearr[fromsw][tosw].append([pid,value])
        weightarr[fromsw][tosw].append([pid,value])

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if len(valuearr[fromsw][tosw])>0:
                valuesum = 0
                for _,value in valuearr[fromsw][tosw]:
                    valuesum += value
                valuetoweight = 1.0/valuesum
                for ipv,pv in enumerate(weightarr[fromsw][tosw]):
                    weightarr[fromsw][tosw][ipv][1] = pv[1] * valuetoweight

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if len(weightarr[fromsw][tosw])>0:
                qweightarr[fromsw][tosw] = quantize_and_adjust_with_indices(weightarr[fromsw][tosw],factor)

    with open(qvarfile,'w') as f:
        for fromsw in range(numsw):
            for tosw in range(numsw):
                if len(qweightarr[fromsw][tosw])>0:
                    for pid,weight in qweightarr[fromsw][tosw]:
                        if weight>0:
                            f.write(f"{fromsw},{tosw},{pid},{weight}\n")

    check(qvarfile, numsw)

In [19]:
trafficnamearr = ["unv1","prv1"]
intervalstart = 4
intervalend = 8
factor = 64
numsw = 80
racke = 0
H = 1

for trafficname in trafficnamearr:
    for interval in range(intervalstart, intervalend):
        ivarfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/ivar_compute_dring_0_0_{trafficname}_{interval-H}_racke{racke}_2_0"
        qivarfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/qivar_dring_0_0_{trafficname}_{interval}_racke{racke}_2_0"
        quantize(ivarfile,qivarfile,factor,numsw)

generate conffile

In [21]:
trafficname = "unv1"
racke=0
multstrarr1 = ["0 2 25","0 17 100","0 1 4","0 17 50","0 21 50","0 51 100","0 59 100","0 34 50","0 38 50","0 17 20"]
multnamearr1 = range(5,51,5)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 4 8 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 4 8 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [22]:
trafficname = "prv1"
multstrarr1 = ["0 2 100","0 5 100","0 7 100","0 10 100","0 12 100","0 15 100","0 17 100","0 20 100","0 22 100","0 25 100","0 37 100","0 49 100","0 62 100","0 74 100"]
multnamearr1 = list(range(2,19,2))+list(range(20,61,10))
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 4 8 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 4 8 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [7]:
trafficname = "cluster_a"
racke=0
multstrarr1 = ["4 8 100","8 17 100","12 25 100","16 34 100","20 42 100","24 51 100","28 59 100","32 67 100","36 76 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [2]:
trafficname = "cluster_b"
racke=0
multstrarr1 = ["0 27 100","0 53 100","0 80 100","1 7 100","1 34 100","1 60 100","1 87 100","2 14 100","2 40 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/u/az6922/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [9]:
trafficname = "cluster_c"
racke=0
multstrarr1 = ["0 19 100","0 38 100","0 57 100","0 75 100","0 94 100","1 13 100","1 32 100","1 51 100","1 70 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

generate sumfile & picklefile

In [ ]:
import pickle

trafficname = "prv1"

homedir = "/home/annzhou"
datadict = dict()
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.conf"
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.txt"
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[32]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}/DRing/src/emp/datacentre/{logfile}",'r') as logf:
                print(logfile)
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")

with open(f'{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.pickle', 'wb') as handle:
    pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)

delete log,run files

In [ ]:
import os

homedir = "/home/annzhou"
dringdir = f"{homedir}/DRing/src/emp/datacentre"
for filedir in os.listdir(dringdir):
    if os.path.isfile(os.path.join(dringdir, filedir)):
        tokens = filedir.split("_")
        if tokens[0]=="m" and (tokens[1]=="leafspine" or tokens[1]=="dring") and tokens[-1].split('.')[-1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[0]=="rrg" or tokens[0]=="leafspine") and tokens[1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[-1]=="run") and ((tokens[0]=="leafspine" and tokens[1]=="leafspine") or (tokens[0]=="rrg" and (tokens[1]=="rrg" or tokens[1]=="dring"))):
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")